In [ ]:
# Cell: Set up Gemini API (Alternative to OpenAI)
import os
with open("config.json") as f:
    config = json.load(f)
os.environ["GOOGLE_API_KEY"] = config["API_KEY"]

# Import Gemini-specific libraries
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
import json

# Create a simple document store (for testing)
documents = [
    "This is a sample document about RAG.",
    "RAG combines retrieval and generation for better answers.",
    "LangChain is a popular framework for building RAG systems."
]

# Split documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Create embeddings and vector store using Gemini
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectorstore = Chroma.from_documents(texts, embeddings)

# Set up the RAG pipeline with Gemini
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())

# Test the RAG system with a sample query
query = "What is RAG?"
result = qa_chain.run(query)
print("Query:", query)
print("Result:", result)


In [ ]:
# Cell: Implement Agentic RAG using Gemini and LangChain's AgentExecutor
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent
from langchain.prompts import StringPromptTemplate
from langchain import LLMChain
from langchain.schema import AgentAction, AgentFinish
import re

# Define a custom prompt template for the agent
class CustomPromptTemplate(StringPromptTemplate):
    template = """Answer the following question as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
{agent_scratchpad}"""

    def format(self, **kwargs):
        # Get the intermediate steps (AgentAction, Observation tuples)
        intermediate_steps = kwargs.pop("agent_scratchpad")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        kwargs["agent_scratchpad"] = thoughts
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in kwargs["tools"]])
        kwargs["tool_names"] = ", ".join([tool.name for tool in kwargs["tools"]])
        return self.template.format(**kwargs)

# Define a custom output parser
class CustomOutputParser:
    def parse(self, llm_output):
        if "Final Answer:" in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        regex = r"Action: (.*?)[\n]*Action Input:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            return AgentFinish(
                return_values={"output": "I could not determine the next action."},
                log=llm_output,
            )
        action = match.group(1).strip()
        action_input = match.group(2).strip(" ").strip('"')
        return AgentAction(tool=action, tool_input=action_input, log=llm_output)

# Define a tool for document retrieval (using the Gemini-based qa_chain)
def retrieve_documents(query):
    return qa_chain.run(query)

tools = [Tool(name="RetrieveDocuments", func=retrieve_documents, description="Useful for retrieving documents related to the query.")]

# Set up the agent with Gemini
prompt = CustomPromptTemplate(template=CustomPromptTemplate.template, input_variables=["input", "agent_scratchpad", "tools"])
output_parser = CustomOutputParser()
llm_chain = LLMChain(llm=llm, prompt=prompt)  # Using the Gemini llm from the previous cell
agent = LLMSingleActionAgent(llm_chain=llm_chain, allowed_tools=[tool.name for tool in tools], output_parser=output_parser)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

# Test the agentic RAG system with Gemini
query = "What is RAG?"
result = agent_executor.run(query)
print("Query:", query)
print("Result:", result)